In [1]:
from functools import lru_cache

import requests
import pandas as pd

headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:66.0) Gecko/20100101 Firefox/66.0"}
REDDIT_ROOT_URL = "https://reddit.com"


def add_json_sufix(url):
    url = REDDIT_ROOT_URL + url if not url.startswith("http") else url
    return url if url.endswith('.json') else url + '.json'


def get_with_headers(url):
    return requests.get(url, headers=headers)


def get_subreddit_posts(subreddit_url):
    print(f"Getting posts from {subreddit_url}...")
    subreddit_url = add_json_sufix(subreddit_url)
    response = get_with_headers(subreddit_url)
    raw_posts = response.json()['data']['children']

    posts = []
    for raw_post in raw_posts:
        post = {}
        raw_post = raw_post['data']
        post['title'] = raw_post['title']
        post['score'] = raw_post['score']
        post['url'] = REDDIT_ROOT_URL + raw_post['permalink']
        post['nsfw'] = raw_post['over_18']

        posts.append(post)

    return posts

In [2]:
subreddit_url = "/r/programming" # https://reddit.com/r/askreddit
posts = get_subreddit_posts(subreddit_url)

Getting posts from /r/programming...


In [4]:
df_posts = pd.DataFrame(posts)
df_posts.head()

,title,score,url,nsfw
0,Bring your monorepo down to size with Git spar...,231,https://reddit.com/r/programming/comments/eqqw...,False
1,"No, dynamic type systems are not inherently mo...",26,https://reddit.com/r/programming/comments/eqv6...,False
2,GCC: C++ coroutines - Initial implementation p...,33,https://reddit.com/r/programming/comments/eqtx...,False
3,What's New in Java 19: The end of Kotlin?,687,https://reddit.com/r/programming/comments/eqgf...,False
4,Algorithm for Drawing Trees,11,https://reddit.com/r/programming/comments/equq...,False


In [5]:
df_posts.shape

(25, 4)